# Процессы

In [1]:
print(5, 6)

5 6


In [2]:
import random
import os
import time
import threading
import multiprocessing
import os

In [3]:
# Типы задач
def cpu_bound_task(n):
    def fib(n):
        if n == 0:
            return 0
        elif n == 1:
            return 1
        return fib(n-1) + fib(n-2)

    print(f'{n} -> {fib(n)}')


def io_bound_task(ind):
    print(f'start task {ind}')
    time.sleep(1)
    print(f'finish task {ind}\n', end='')

In [4]:
%%time

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    cpu_bound_task(number)

36 -> 14930352
35 -> 9227465
34 -> 5702887
33 -> 3524578
32 -> 2178309
31 -> 1346269
30 -> 832040
25 -> 75025
20 -> 6765
15 -> 610
10 -> 55
CPU times: user 42.4 s, sys: 42.4 ms, total: 42.5 s
Wall time: 42.6 s


In [5]:
%%time

# CPU bound задача на потоках
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = threading.Thread(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

20 -> 6765
15 -> 610
10 -> 55
25 -> 75025
30 -> 832040
31 -> 1346269
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
CPU times: user 1min 4s, sys: 844 ms, total: 1min 5s
Wall time: 1min 6s


In [6]:
%%time

# CPU bound задача на процессах
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = multiprocessing.Process(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()


15 -> 610
20 -> 6765
10 -> 55
25 -> 75025
30 -> 832040
31 -> 1346269
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
CPU times: user 32.6 ms, sys: 52.7 ms, total: 85.3 ms
Wall time: 20.3 s


In [ ]:
%%time

for ind in range(5):
    io_bound_task(ind)

In [ ]:
%%time

# IO bound задача на потоках
tasks = []

for ind in range(5):
    task = threading.Thread(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

In [ ]:
%%time

# IO bound задача на процессах
tasks = []

for ind in range(5):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

In [ ]:
# Получение идентификатора процесса
def io_bound_task(ind):
    print(f'start task {ind} with {os.getpid()}')
    time.sleep(3)
    print(f'finish task {ind}')
    
tasks = []

for ind in range(10):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

In [7]:
# Пулл процессов
from multiprocessing import Pool


def getpid(n):
    time.sleep(2)
    return os.getpid()

with Pool(3) as p:
    print(p.map(getpid, range(5)))

[4377, 4378, 4379, 4378, 4377]


In [10]:
# Шаринг ресурсов
share_memory = {
    'count': 0,
}
def share_memory_task():
    print(f'read {share_memory["count"]}')
    share_memory['count'] += 2
    print(f'write {share_memory["count"]}')

tasks = []

for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

read 0
write 2
read 0
read 0
write 2
write 2
read 0
read 0
read 0
write 2
write 2
write 2
read 0
read 0
read 0
read 0
write 2
write 2
write 2
write 2


In [ ]:
# Шаринг ресурсов через файл
filename = 'share_memory.tmp'

with open(filename, 'w') as fd:
    fd.write('0')

def share_memory_task():
    with open(filename) as fd:
        count = int(fd.read())
    print(f'read {count}')

    with open(filename, 'w') as fd:
        fd.write(str(count + 1))
    print(f'write {count + 1}')


tasks = []

for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

with open(filename) as fd:
    print(f'COUNT: {fd.read()}')

In [12]:
from multiprocessing import Manager, Queue


PROCESSES_COUNT = 5
to_process_list = [
    [1, 2, 3], [4, 5, 6], [7, 8, 9],
    [10, 11, 12], [13, 14, 15],
    [16, 17, 18], [19, 20, 21],
    [22, 23, 24], [25, 26, 27]
]


def func(x):
    time.sleep(3)
    return 1 << x


def producer_func(to_process, queue):
    for elem in to_process:
        res = func(elem)
        queue.put(res)
        print(f"Hey, I've just put {res} to queue!")


def consumer_func(queue):
    while True:
        res = queue.get()
        if res == 'kill':
            break
        print(f"Wow, I found {res} in queue!")


def do_work(to_process_list):
    manager = Manager()
    queue = manager.Queue()
    pool = Pool(PROCESSES_COUNT)

    pool.apply_async(consumer_func, (queue, ))

    jobs = []
    for to_process in to_process_list:
        job = pool.apply_async(producer_func, (to_process, queue))
        jobs.append(job)

    for job in jobs:
        job.get()

    queue.put('kill')
    print('kill')
    pool.close()
    pool.join()
    

do_work(to_process_list)

Hey, I've just put 2 to queue!
Hey, I've just put 16 to queue!
Wow, I found 2 in queue!
Wow, I found 16 in queue!
Hey, I've just put 1024 to queue!
Wow, I found 1024 in queue!
Wow, I found 128 in queue!
Hey, I've just put 128 to queue!
Wow, I found 4 in queue!
Hey, I've just put 256 to queue!
Hey, I've just put 4 to queue!
Hey, I've just put 32 to queue!
Hey, I've just put 2048 to queue!
Wow, I found 256 in queue!
Wow, I found 2048 in queue!
Wow, I found 32 in queue!
Hey, I've just put 8 to queue!
Hey, I've just put 512 to queue!
Wow, I found 8 in queue!
Hey, I've just put 4096 to queue!
Hey, I've just put 64 to queue!
Wow, I found 512 in queue!
Wow, I found 64 in queue!
Wow, I found 4096 in queue!
Wow, I found 524288 in queue!
Hey, I've just put 8192 to queue!
Hey, I've just put 524288 to queue!
Hey, I've just put 65536 to queue!
Wow, I found 8192 in queue!
Hey, I've just put 4194304 to queue!
Wow, I found 65536 in queue!
Wow, I found 4194304 in queue!
Hey, I've just put 131072 to que